In [8]:
import requests

response= requests.get("http://api.open-notify.org/iss-now.json")


print(response) 
print(response.status_code)

<Response [200]>
200


In [6]:
response= requests.get("http://api.open-notify.org/iss-now.jsonaaaaaaa")
print(response.raise_for_status())  # 에러코드의 종류와 뜻을 알려줌

HTTPError: 404 Client Error: Not Found for url: http://api.open-notify.org/iss-now.jsonaaaaaaa

## #오류의 종류
- 1xx : 처리중이니 기다려라
- 2xx : 처리ok
- 3xx : 너한테 권한이 없으니 가라
- 4xx : 너는 잘못되었거나 찾는게 없다
- 5xx : 서버가 잘못되었다
- https://www.webfx.com/web-development/glossary/http-status-codes/

In [20]:
# iss 가 지금 있는 위치

response= requests.get("http://api.open-notify.org/iss-now.json")

data = response.json()
print(data)

data1 = response.json()["iss_position"]
print(data1)

longitude = response.json()["iss_position"]["longitude"]
print(longitude)
latitude = response.json()["iss_position"]["latitude"]
print(latitude)

iss_position = (longitude, latitude) # 경도와 위도
print(iss_position)

{'message': 'success', 'iss_position': {'longitude': '-109.5579', 'latitude': '39.0403'}, 'timestamp': 1668844630}
{'longitude': '-109.5579', 'latitude': '39.0403'}
-109.5579
39.0403
('-109.5579', '39.0403')


# #칸예 명언 api로 가져오기

In [26]:
import requests
response= requests.get("https://api.kanye.rest/")
response.raise_for_status()
data = response.json()
quote = data["quote"] # 명언

print(quote) # 실행할때마다 다른 명언이 나옴

I still think I am the greatest.


## ui 만들어서 적용해보기

In [2]:
from tkinter import *
import requests


def get_quote():
    response= requests.get("https://api.kanye.rest/")
    response.raise_for_status()
    data = response.json()
    quote = data["quote"]
    canvas.itemconfig(quote_text, text=quote)

window = Tk()
window.title("Kanye Says...")
window.config(padx=50, pady=50)

canvas = Canvas(width=300, height=414)
background_img = PhotoImage(file="background_33.png")
canvas.create_image(150, 207, image=background_img)
quote_text = canvas.create_text(150, 207, text="Kanye Quote Goes HERE", width=250, font=("Arial", 30, "bold"), fill="white")
canvas.grid(row=0, column=0)

kanye_img = PhotoImage(file="kanye_33.png")
kanye_button = Button(image=kanye_img, highlightthickness=0, command=get_quote)
kanye_button.grid(row=1, column=0)



window.mainloop()

# #일출과 일몰 시간 알아내기

In [42]:
import requests
from datetime import datetime

MY_LAT = 44.781950 
MY_LONG = -76.808130

parameters = {
    "lat" : MY_LAT,
    "lng" : MY_LONG,
    "formatted" : 0,
}

response = requests.get(url="https://api.sunrise-sunset.org/json")
response.raise_for_status()
data = response.json()

sunrise = data["results"]["sunrise"].split(":")[0]+" "+data["results"]["sunrise"].split(":")[2].split()[1]
sunset = data["results"]["sunset"].split(":")[0]+" "+data["results"]["sunset"].split(":")[2].split()[1]

time_now = datetime.now()

print("0"+sunrise)
print("0"+sunset)
print(time_now.strftime("%I %p"))

05 AM
05 PM
03 PM


In [39]:
type(sunset)

str

In [43]:
# iss api 시각 변경되기 전
sunrise = data["results"]["sunrise"].split("T")[1].split(":")[0]
sunset = data["results"]["sunset"].split("T")[1].split(":")[0]
# print(sunrise.split("T")[1].split(":")[0])
# print(sunset.split("T")[1].split(":")[0])

print(sunrise)
print(sunset)
print(time_now.hour)

22
08
17


## 60초마다 확인해서 ISS가 내 위치에 있고 밤이면 메일보내기

In [ ]:
import requests
from datetime import datetime
import smtplib
import time

my_e = "test@naver.com"
my_p = "abc123"
MY_LAT = 44.781950 
MY_LONG = -76.808130

def is_iss_overhead():
    response = requests.get(url="http://api.open-notify.org/iss-now.json")
    response.raise_for_status()
    data = response.json()

    iss_latitude = float(data["iss_position"]["latitude"])
    iss_longitude = float(data["iss_position"]["longitude"])

    if MY_LAT-5 <= iss_latitude <= MY_LAT+5 and MY_LONG-5 <= iss_latitude <= MY_LONG+5:
        return True

def is_night():
    parameters = {
        "lat": MY_LAT,
        "lng": MY_LONG,
        "formatted": 0,
    }

    response = requests.get("https://api.sunrise-sunset.org/json", params=parameters)
    response.raise_for_status()
    data = response.json()
    sunrise = int(data["results"]["sunrise"].split(":")[0])
    sunset = int(data["results"]["sunset"].split(":")[0])

    time_now = int(time_now.strftime("%I"))
    
    if time_now >= sunset or time_now <= sunrise:
        return True

while True:
    time.sleep(60)
    if is_iss_overhead() and is_night():
        connection = smtplib.SMTP("smtp.naver.com")
        connection.starttls()
        connection.login(my_e, my_p)
        connection.sendmail(from_addr=my_e, to_addrs=my_e,
                            msg="Subject:Look up\n\n위에 iss가 지나가고 있습니다.")